# Meeting 20221026
- **Excute the bootstrap result without selecting well logs**
- **Try Kirging water level data**
- **Compare corrected water level data with gamma water level data**


Initial setup

In [ ]:
import os
import numpy as np
import tTEM_toolbox as tt
import pandas as pd
import plotly.offline as po
import plotly.graph_objects as go
import plotly.express as px
from pyproj import Transformer
import pykrige as pk
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
#import importlib
#importlib.reload(tt.bootstrap)

In [ ]:
os.chdir(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\well_Utah')
well_info = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\well_Utah\USGSdownload\NWISMapperExport.xlsx'
location = r"C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\Gamma\location.csv"
welllog = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\Plot_with_well_log\Well_log.xlsx'
elevation = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\well_Utah\usgs_water_elevation.csv'
ttemname = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\Plot_with_well_log\PD1_I01_MOD.xyz'
ttemname2 = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\Plot_with_well_log\PD22_I03_MOD.xyz'
DOI = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\Plot_with_well_log\DOID1_DOIStaE.xyz'
time = '2022-3'
def splt_ttem(ttem_df, gwsurface_result):
    def get_distance(group1, group2):
        dis = np.sqrt((group1[0] - group2[0]) ** 2 + (group1[1] - group2[1]) ** 2)
        return dis
    abv_water_table = []
    blw_water_table = []
    ttem_groups = ttem_df.groupby(['UTMX', 'UTMY'])
    well_location = gwsurface_result[['UTMX', 'UTMY']].values
    for name, group in ttem_groups:
        ttem_xy = list(group[['UTMX', 'UTMY']].iloc[0])
        ttem_well_distance = list(map(lambda x: get_distance(ttem_xy, x), well_location))
        match = gwsurface_result.iloc[ttem_well_distance.index(min(ttem_well_distance))]
        elevation = match['water_elevation']
        ttem_abv = group[group['Elevation_End'] >= elevation]
        abv_water_table.append(ttem_abv)
        ttem_blw = group[group['Elevation_End'] < elevation]
        blw_water_table.append(ttem_blw)

    ttem_above = pd.concat(abv_water_table)
    ttem_below = pd.concat(blw_water_table)
    return ttem_above, ttem_below
def plot_bst(dataframe):
    """
    plot bootstrap result

    :param dataframe:
    :return: plotly fig
    """
    fig_hist = go.Figure()
    fig_hist.data = []
    fig_hist.add_trace(go.Histogram(x=dataframe.fine, name='Fine', marker_color='Blue', opacity=0.75))
    fig_hist.add_trace(go.Histogram(x=dataframe.coarse, name='Coarse', marker_color='Red', opacity=0.75))
    if dataframe.mix.sum() == 0:
        print("skip plot mix because there is no data")
    else:
        fig_hist.add_trace(go.Histogram(x=dataframe.mix, name='Mix', marker_color='Green', opacity=0.75))
    fig_hist.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')

    return fig_hist

In [ ]:
water = tt.main.GWSurface(waterwell=well_info)

In [ ]:
water_format = water.format(elevation=None, time=time)
water_format13_3 =  water.format(elevation=None, time='2013-3')
water_format12_11 = water.format(elevation=None, time='2012-11')
elevation_diff12_13 = water_format12_11['sl_lev_va2012-11'].subtract(water_format13_3['sl_lev_va2013-3'])
#water_format.dropna(inplace=True)
water_format.to_csv(r'D:\demo.csv')
water_format['water_elevation'] = water_format['sl_lev_va2022-3']

In [ ]:
water_diff_krig = pk.ok.OrdinaryKriging(water_format["UTMX"], water_format['UTMY'],elevation_diff12_13)

In [ ]:
water_kirg = pk.ok.OrdinaryKriging(water_format["UTMX"], water_format['UTMY'],water_format['sl_lev_va2022-3'])
x = np.arange(np.min(water_format['UTMX'])-5000, np.max(water_format['UTMX'])+5000,500)
y = np.arange(np.min(water_format['UTMY'])-5000, np.max(water_format['UTMY'])+5000,500)
z,ss = water_kirg.execute('grid',x, y)
z_diff, ss_diff = water_diff_krig.execute('grid',x,y)
x_grid,y_grid=np.meshgrid(x,y)
x_flat = x_grid.flatten()
y_flat = y_grid.flatten()
z_flat = (z+z_diff).flatten()
water_kirg_org = pd.DataFrame({'UTMX':x_flat,'UTMY':y_flat,'water_elevation':z.flatten()})
water_corrected = pd.DataFrame({'UTMX':x_flat,'UTMY':y_flat,'water_elevation':z_flat})

In [ ]:
fig = go.Figure()
trace = go.Scatter3d(x=x_flat,y=y_flat,z=z_flat)
fig.add_trace(trace)
trace_org = tt.plot.generate_trace(water_format,'water')
init_notebook_mode(connected = True)
fig.add_trace(trace_org)
fig.update_layout(scene_aspectmode='data')
                  #scene_aspectratio=dict(x=1,y=1,z=0.3))

In [ ]:
from pyproj import Transformer
transformer = Transformer.from_crs('epsg:4326', 'epsg:32612')
location = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\Gamma\location.csv'
location_df = pd.read_csv(location)
long = location_df['X']
lat = location_df['Y']
x, y = transformer.transform(lat, long)
trace_gamma = go.Scatter3d(x=x,y=y,z=location_df['water_level'])
fig.add_trace(trace_gamma)
fig.update_layout(scene_aspectmode='data')

In [ ]:
ttem = tt.main.ProcessTTEM(ttem_path=[ttemname, ttemname2],
                                   welllog=welllog,
                                   DOI_path=DOI,
                                   layer_exclude=[])
data = ttem.data()
ttem_match, well_match = tt.bootstrap.select_closest(data,
                                                     tt.process_well.format_well(welllog, upscale=100),distance=1000,
                                                       showskip=False)
ttem_above_wt, ttem_below_wt = splt_ttem(data, water_corrected)
bootstrap_above = tt.main.ProcessTTEM(ttem_path=ttem_above_wt, welllog=welllog)
bootstrap_above_result = bootstrap_above.ttem_well_connect(distance=1000)
bootstrap_below = tt.main.ProcessTTEM(ttem_path=ttem_below_wt, welllog=welllog)
bootstrap_below_result = bootstrap_below.ttem_well_connect(distance=1000)

In [ ]:
ttem_above_match, well_match = tt.bootstrap.select_closest(ttem_above_wt,tt.process_well.format_well(welllog, upscale=100),distance=1000,showskip=False)
ttem_below_match, well_match = tt.bootstrap.select_closest(ttem_below_wt,tt.process_well.format_well(welllog, upscale=100),distance=1000,showskip=False)

In [ ]:
ttem_above_match.shape

In [ ]:
ttem_below_match.shape

In [ ]:
import importlib; importlib.reload(tt.Rock_trans)
res_conf_df = tt.bootstrap.packup(bootstrap_above_result[1]['fine'],bootstrap_above_result[1]['mix'], bootstrap_above_result[1]['coarse'] )
rk_trans = tt.Rock_trans.rock_transform(bootstrap_above_result[0], res_conf_df)

In [ ]:
res_conf_df

In [ ]:
import importlib
importlib.reload(tt.plot)
trace_ttem = tt.plot.generate_trace(bootstrap_above.data(),'ttem')
#fig.add_trace(trace_ttem)
fig.show()

In [ ]:
fig_hist_ab=plot_bst(bootstrap_above_result[1])
fig_hist_ab.show(renderer='browser')

In [ ]:
fig_hist_bl=plot_bst(bootstrap_below_result[1])
fig_hist_bl.show(renderer='browser')

In [ ]:
ttem_above_wt.Resistivity.plot.hist()

In [ ]:
ttem_below_wt.Resistivity.plot.hist()

In [ ]:
#Rock transform
colorRes = [[0, 'rgb(0,0,190)'],
            [1 / 16, 'rgb(0,75,220)'],
            [2 / 16, 'rgb(0,150,235)'],
            [3 / 16, 'rgb(0,200,255)'],
            [4 / 16, 'rgb(80,240,255)'],
            [5 / 16, 'rgb(30,210,0)'],
            [6 / 16, 'rgb(180,255,30)'],
            [7 / 16, 'rgb(255,255,0)'],
            [8 / 16, 'rgb(255,195,0)'],
            [9 / 16, 'rgb(255,115,0)'],
            [10 / 16, 'rgb(255,0,0)'],
            [11 / 16, 'rgb(255,0,120)'],
            [12 / 16, 'rgb(140,40,180)'],
            [13 / 16, 'rgb(165,70,220)'],
            [14 / 16, 'rgb(195,130,240)'],
            [15 / 16, 'rgb(230,155,240)'],
            [1, 'rgb(230,155,255)']]
colorrock = [[0, 'rgb(30,144,255)'],
                 [0.5, 'rgb(255,255,0)'],
                 [1,'rgb(255,0,0)']]

In [ ]:
line100 = data[data['Line_No'] == 100]
line100 = line100[(line100['UTMX'] <= 350118.90) & (line100['UTMX']>349221.50)]
line100['distance'] = np.sqrt(line100['UTMX'].copy()**2 + line100['UTMY'].copy()**2)
line100['distance'] = line100['distance'] - line100['distance'].min()
grid_y, grid_x = np.mgrid[line100['Elevation_Cell'].max():line100['Elevation_Cell'].min():690j, line100['distance'].min():line100['distance'].max():3120j]
grid_z1 = np.full((690,3120),np.nan)
groups = line100.groupby('distance')
for name, group in groups:
    x_index = np.where((grid_x>=name-1) & (grid_x <=name+1))[1][0]
    for index, line in group.iterrows():
        y_index = np.where((grid_y[:,x_index] <= line['Elevation_Cell']) & (grid_y[:,x_index]>=line['Elevation_End']))[0]
        grid_z1[y_index[0]-2:y_index[-1]+2,x_index-3:x_index+3] = np.log10(line['Resistivity'])
fig=px.imshow(grid_z1,range_color=(0,3),color_continuous_scale=colorRes)
fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')
fig.update_layout(yaxis=dict(title='Elevation (m)',tickmode='array',tickvals=[0,100,200,300,400,500,600]
                             ,ticktext=['1779','1767.5','1756','1744.5','1733','1721.5','1710']),
                  xaxis=dict(title='Distance (m)'))
fig.update_coloraxes(colorbar=dict(ticks='outside',title='Resistivity', tickvals=[0,1,2,3], ticktext=['1','10','100','1000'],tickmode='array',len=0.5))
fig.show(renderer='browser')
fig_ttem = go.Figure(trace_ttem)
fig_ttem.add_trace(trace)
fig_ttem.show(renderer='browser')

In [ ]:
rock100 = rk_trans[rk_trans['Line_No'] == 100]
rock100 = rock100[(rock100['UTMX'] <= 350539.30) & (rock100['UTMX']>349221.50)]
rock100['distance'] = np.sqrt(rock100['UTMX'].copy()**2 + rock100['UTMY'].copy()**2)
rock100['distance'] = rock100['distance'] - rock100['distance'].min()
grid_y, grid_x = np.mgrid[rock100['Elevation_Cell'].max():rock100['Elevation_Cell'].min():690j, rock100['distance'].min():rock100['distance'].max():3120j]
grid_z1 = np.full((690,3120),np.nan)
groups = rock100.groupby('distance')
for name, group in groups:
    x_index = np.where((grid_x>=name-1) & (grid_x <=name+1))[1][0]
    for index, line in group.iterrows():
        y_index = np.where((grid_y[:,x_index] <= line['Elevation_Cell']) & (grid_y[:,x_index]>=line['Elevation_End']))[0]
        grid_z1[y_index[0]-2:y_index[-1]+2,x_index-3:x_index+3] = line['Identity_n']
fig_rock=px.imshow(grid_z1, range_color=(1,3),color_continuous_scale=colorrock)
fig_rock.show(renderer='browser')

In [ ]:
fig.show(renderer='browser')

In [ ]:
trace=tt.plot.generate_trace(line100,'ttem')
line100 = data[data['Line_No'] == 100]
line100 = line100[(line100['UTMX'] <= 350539.30) & (line100['UTMX']>349221.50)]
water_level = []
groups = line100.groupby(['UTMX','UTMY'])
for name, group in groups:
    water_corrected['distance'] = np.sqrt((water_corrected['UTMX']-name[0])**2+(water_corrected['UTMY']-name[1])**2)
    water_level.append(water_corrected[water_corrected['distance']==water_corrected['distance'].min()])
filter_water_elevation = pd.concat(water_level)
fig_water = go.Scatter3d(x=filter_water_elevation['UTMX'],y=filter_water_elevation['UTMY'],z=filter_water_elevation['water_elevation'])
fig = go.Figure(trace)
fig.add_trace(fig_water)
fig.show(renderer='browser')

In [ ]:
water_corrected

In [ ]:
filter_water_elevation